In [2]:
# Import required directories

import os
import gzip
import pandas as pd

In [ ]:
# Set the directory containing the .gz files to unzip the dowloaded .gz files
dir_path = '../CONTRIBUTING.mddata/raw'

# Loop through each .gz file in the directory
for filename in os.listdir(dir_path):
    if filename.endswith('.gz'):
        
        # Open .gz file and extract contents to a new file
        with gzip.open(os.path.join(dir_path, filename), 'rb') as f_in:
            with open(os.path.join(dir_path, filename[:-3]), 'wb') as f_out:
                f_out.write(f_in.read())

In [ ]:
# Read json files in batch
# Set the directory containing the JSON files
dir_path = '../data/raw'

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each JSON file in the directory
for filename in os.listdir(dir_path):
    if filename.endswith('.json'):
        
        # Read JSON file into a DataFrame
        with open(os.path.join(dir_path, filename), 'r') as f:
            df = pd.read_json(f, lines=True)
        
        # Append DataFrame to list
        dfs.append(df)
        print('file:', filename, ' Done!')

# Concatenate all DataFrames in the list into one DataFrame
final_df = pd.concat(dfs, ignore_index=True)

In [18]:
#final_df = pd.read_csv('../data/raw/1.csv', index_col=0)
final_df = pd.read_json('../data/raw/2023-03-17-23.json', lines=True)

In [19]:
# Drop unnecessary columns
drop_df = final_df.drop(['payload', 'public', 'org'], axis=1)

In [20]:
# Some data cleaning and joining
expanded_actor_df = pd.json_normalize(drop_df['actor'])
expanded_actor_df = expanded_actor_df.rename(columns={'id':'actor_id',
                                                      'url':'actor_url'})
actor_df = expanded_actor_df.drop(['login', 'display_login', 'gravatar_id',
                                   'actor_url', 'avatar_url'], axis=1)

expanded_repo_df = pd.json_normalize(drop_df['repo'])
expanded_repo_df = expanded_repo_df.rename(columns={'id':'repo_id',
                                                    'url':'repo_url'})
repo_df = expanded_repo_df.drop(['name', 'repo_url'], axis=1)

In [21]:
# Export the final raw df
df_main = drop_df.reset_index()
df_final = df_main.drop(['actor', 'repo', 'index'], axis=1)

merged_df_final = pd.concat([df_final,
                             actor_df,
                             repo_df], axis=1)
merged_df_final.to_csv('../data/raw/raw_data-23.csv')

In [2]:
pds = []
for i in range(0, 24):
    name = '../data/raw/raw_data-'+str(i)+'.csv'
    df = pd.read_csv(name, index_col = 0)
    pds.append(df)

In [3]:
df_raw = pd.concat(pds, ignore_index=True)

In [3]:
df_raw = pd.read_csv('../data/raw/raw_data-main.csv', index_col=0)

In [4]:
df_raw = df_raw.sample(n=2000000)

In [5]:
df_raw.to_csv('../data/raw/raw_data.csv')